In [12]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("[INST]What is the meaning of {word}?[/INST]")

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={
    "max_new_tokens": 250,
    }
)
llm.client.api_url = 'https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1'

chain = prompt | llm

chain.invoke({"word": "potato"})

' A potato is a starchy vegetable that is grown underground in rows in a shallow, well-drained trench. It is a tuber, which means it develops underground from the stem of the plant. Potatoes are a popular food source and are used in many different dishes, including soups, salads, stews, and baked dishes. They are also used to make potato chips and other snacks. Potatoes are high in carbohydrates and are a good source of fiber, vitamin C, and several B vitamins.'